In [15]:
import pandas as pd
import numpy as np
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from sklearn import preprocessing

from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV 
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import LeaveOneOut

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

from sklearn import metrics

from scipy.stats import sem, t
from scipy import mean

#Oversampling
# from imblearn.over_sampling import RandomOverSampler
from kmeans_smote import KMeansSMOTE


### multiprocessing
from multiprocessing.pool import Pool


In [16]:
data = pd.read_csv('prepared Data.csv')
print(data.shape)
data.head()


(922, 33)


,gender,age,SBP,BMI,etiology of CKD,Hb,Alb,Cr,eGFR,CKD_stage,...,eGFR(6M),eGFR(12M),eGFR(18M),eGFR(24M),eGFR(30M),eGFR(36M),eGFR(last visit),average_obs,obsevasion_ duration,fclass
0,2,74,120.0,23.137669,2,12.0,4.0,1.20,89.981926,3,...,26.454698,24.331582,24.682189,21.614854,20.420524,20.420524,18.495328,25.275139,37,0
1,1,57,139.0,28.515625,2,15.9,4.8,0.84,88.330020,2,...,78.287758,71.343858,72.845992,71.908942,71.562914,67.225032,67.225032,72.392152,37,0
2,1,32,154.0,24.582701,4,14.4,4.4,0.87,86.973875,2,...,75.027238,69.595257,68.856399,72.901926,69.749275,69.171408,69.171408,72.694258,36,0
3,1,60,144.0,30.737407,2,14.4,4.7,2.22,86.874201,4,...,26.885061,24.917353,28.581660,29.237135,25.556002,25.183703,25.183703,26.485251,35,0
4,1,49,135.0,23.758726,2,17.0,4.1,1.39,86.782629,3,...,46.978867,45.829455,41.488436,41.801561,38.106104,38.106104,38.106104,43.081581,30,0


In [17]:
col = data.columns

In [18]:
count = Counter(data['fclass'])
count


Counter({0: 611, 1: 170, 5: 51, 2: 42, 6: 48})

# Feature Selection

In [19]:
# import pandas as pd
# import numpy as np
# data = pd.read_csv("D://Blogs//train.csv")
# X = data.iloc[:,0:20]  #independent columns
# y = data.iloc[:,-1]    #target column i.e price range
from sklearn.ensemble import ExtraTreesClassifier
importances = dict()
for col in data.iloc[:,:-1].columns:
    importances[col] = 0

for i in range(50):
    model = ExtraTreesClassifier()
    model.fit(data.iloc[:,:-1],data['fclass'])
    matplotlib.rcParams['figure.figsize'] = (8.0, 6.0)
    # print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
    #plot graph of feature importances for better visualization
    feat_importances = pd.Series(model.feature_importances_, index=data.iloc[:,:-1].columns)
    for col in data.iloc[:,:-1].columns:
        importances[col] += dict(feat_importances)[col]
#     feat_importances.nlargest(5).plot(kind='barh')
#     plt.show()
    
newpd = pd.DataFrame()
for col in data.iloc[:,:-1].columns:
    newpd[col] = list([importances[col]])
newpd.to_csv('feature_importamce.csv', index=False)

# Tuning

In [22]:
#preprocessing
max_abs_scaler = preprocessing.MaxAbsScaler()
X_train = max_abs_scaler.fit_transform(data.values[:,:-1])
y_train = data.values[:,-1]

# kmeans_smote = KMeansSMOTE(
#     kmeans_args={
#         'n_clusters': 63
#     },
#     smote_args={
#         'k_neighbors': 10
#     })
# X_train, y_train = kmeans_smote.fit_sample(X_train, data.values[:,-1])


In [23]:
'Support Vector Machine'

# defining parameter range for SVM
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['poly','rbf','sigmoid', 'linear']}  
  
grid = GridSearchCV(SVC(), param_grid, n_jobs=8,refit = True, verbose = 3, cv=5) 
 
# fitting the model for grid search 

grid.fit(X_train , y_train)
print('Mean Accuracy: %.3f' % grid.best_score_)
print('Config: %s' % grid.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    3.2s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:  2.6min finished


{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [24]:
'Random Forest'
## Using grid search
rf = RandomForestClassifier()
# Number of trees in random forest
n_estimators = list(range(1,61,15))
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = list(range(10, 60, 10))
# Minimum number of samples required to split a node
min_samples_split = [2, 4, 6, 8, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
}

grid = GridSearchCV(estimator = rf, param_grid = random_grid, n_jobs=8, refit = True, verbose = 3, cv=5) 
 
# fitting the model for grid search 
grid.fit(X_train , y_train)
grid.best_params_
print('Mean Accuracy: %.3f' % grid.best_score_)
print('Config: %s' % grid.best_params_)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 800 candidates, totalling 4000 fits


[Parallel(n_jobs=8)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 208 tasks      | elapsed:    3.6s
[Parallel(n_jobs=8)]: Done 528 tasks      | elapsed:    9.0s
[Parallel(n_jobs=8)]: Done 976 tasks      | elapsed:   16.8s
[Parallel(n_jobs=8)]: Done 1552 tasks      | elapsed:   26.9s
[Parallel(n_jobs=8)]: Done 2256 tasks      | elapsed:   39.4s
[Parallel(n_jobs=8)]: Done 3088 tasks      | elapsed:   54.1s


Mean Accuracy: 0.811
Config: {'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 31}


[Parallel(n_jobs=8)]: Done 4000 out of 4000 | elapsed:  1.2min finished


In [25]:
'Gaussian Naive Bayes'

nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
print(params_NB)

gs_NB = GridSearchCV(estimator=nb_classifier,
                     n_jobs=8,
                 refit = True,
                 param_grid=params_NB, 
                 cv=5,   # use any cross validation technique 
                 verbose=3) 

gs_NB.fit(X_train, y_train)

print('Mean Accuracy: %.3f' % gs_NB.best_score_)
print('Config: %s' % gs_NB.best_params_)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s


{'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02, 2.31012970e-02, 1.87381742e-02,
       1.51991108e-02, 1.23284674e-02, 1.00000000e-02, 8.11130831e-03,
       6.57933225e-03, 5.33669923e-03, 4.32876128e-03, 3.51119173e-03,
       2.84803587e-03, 2.31012970e-03, 1.87381742e-03, 1.51991108e-03,
       1.23284674e-03, 1.00000000e-03, 8.11130831e-04, 6.57933225e-04,
       5.33669923e-04, 4.32876128e-04, 3.51119173e-04, 2.84803587e-04,
       2.31012970e-04, 1.87381742e-04, 1.51991108e-04, 1.23284674e-04,
       1.00000000e-04, 8.11130831e-05, 6.57933225e-05, 5.33669923e-05,
       4.32876128e-05, 3.51119173e-05, 2.84803587e-05, 2.31012970e-05,
       1.87381742e-05, 1.51991108e-05, 1.23284674e-05, 1.00

[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.5s finished


{'var_smoothing': 1.0}

In [35]:
print('Mean Accuracy: %.3f' % gs_NB.best_score_)
print('Config: %s' % gs_NB.best_params_)

Mean Accuracy: 0.715
Config: {'var_smoothing': 1.0}


In [26]:
'Linear Discriminant Analysis'
from numpy import arange
# define grid
grid = dict()
grid['solver'] = ['svd', 'lsqr', 'eigen']
grid['shrinkage'] = arange(0, 1, 0.01)
# define model
model = LinearDiscriminantAnalysis()
LDA_search = GridSearchCV(model, grid,
                          n_jobs=8,
                 refit = True,
                 cv=5,  
                 verbose=3) 
LDA_search.fit(X_train, y_train)
# summarize
print('Mean Accuracy: %.3f' % LDA_search.best_score_)
print('Config: %s' % LDA_search.best_params_)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  17 tasks      | elapsed:    0.1s


Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=8)]: Done 1008 tasks      | elapsed:    1.4s


Mean Accuracy: 0.778
Config: {'shrinkage': 0.03, 'solver': 'lsqr'}


[Parallel(n_jobs=8)]: Done 1500 out of 1500 | elapsed:    2.2s finished


In [27]:
'Decision Tree'
DT = DecisionTreeClassifier()

criterion = ['gini', 'entropy']
max_depth =  list(range(10, 60, 5))
max_features = list(range(1 ,X_train.shape[1]))
min_samples_leaf = list(range(1, 31, 3))
min_samples_split = list(range(2, 10, 2))
presort = [False, True]

DT_param = {'criterion': criterion,
               'max_depth': max_depth,
               'max_features': max_features,
               'min_samples_leaf': min_samples_leaf,
               'min_samples_split': min_samples_split}

DT_grid = GridSearchCV(estimator = DT, param_grid = DT_param,n_jobs=8, refit = True, verbose = 3, cv=5) 
# fitting the model for grid search 
DT_grid.fit(X_train , y_train)
DT_grid.best_params_
# summarize
print('Mean Accuracy: %.3f' % DT_grid.best_score_)
print('Config: %s' % DT_grid.best_params_)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s


Fitting 5 folds for each of 24800 candidates, totalling 124000 fits


[Parallel(n_jobs=8)]: Done 2032 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done 9040 tasks      | elapsed:   12.7s
[Parallel(n_jobs=8)]: Done 19792 tasks      | elapsed:   29.2s
[Parallel(n_jobs=8)]: Done 33616 tasks      | elapsed:   49.6s
[Parallel(n_jobs=8)]: Done 50512 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done 67280 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done 77040 tasks      | elapsed:  2.2min
[Parallel(n_jobs=8)]: Done 86976 tasks      | elapsed:  2.7min
[Parallel(n_jobs=8)]: Done 98816 tasks      | elapsed:  3.2min
[Parallel(n_jobs=8)]: Done 111648 tasks      | elapsed:  3.8min
[Parallel(n_jobs=8)]: Done 124000 out of 124000 | elapsed:  4.3min finished


Mean Accuracy: 0.807
Config: {'criterion': 'gini', 'max_depth': 40, 'max_features': 18, 'min_samples_leaf': 25, 'min_samples_split': 4}


In [28]:
'K Neighbors Classifier'
  
## Using grid search
KNC = KNeighborsClassifier()
metrics = ['euclidean','manhattan'] 
neighbors = np.arange(1, 20)
distances = list(range(1,10,2))
leaf_size = list(range(1,30,2))

# Create the random grid
KNC_param = {'metric': metrics,
               'n_neighbors': neighbors,
               'p': distances,
               'leaf_size': leaf_size
}

KNC_grid = GridSearchCV(estimator = KNC, param_grid = KNC_param, n_jobs=8, refit = True, verbose = 3, cv=5) 
 
# fitting the model for grid search 
KNC_grid.fit(X_train , y_train)
KNC_grid.best_params_
print('Mean Accuracy: %.3f' % KNC_grid.best_score_)
print('Config: %s' % KNC_grid.best_params_)

Fitting 5 folds for each of 2850 candidates, totalling 14250 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 208 tasks      | elapsed:    2.6s
[Parallel(n_jobs=8)]: Done 528 tasks      | elapsed:    7.9s
[Parallel(n_jobs=8)]: Done 976 tasks      | elapsed:   16.5s
[Parallel(n_jobs=8)]: Done 1552 tasks      | elapsed:   21.5s
[Parallel(n_jobs=8)]: Done 2256 tasks      | elapsed:   28.0s
[Parallel(n_jobs=8)]: Done 3264 tasks      | elapsed:   36.6s
[Parallel(n_jobs=8)]: Done 5184 tasks      | elapsed:   50.2s
[Parallel(n_jobs=8)]: Done 7360 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done 9792 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done 12480 tasks      | elapsed:  1.6min


Mean Accuracy: 0.764
Config: {'leaf_size': 1, 'metric': 'manhattan', 'n_neighbors': 18, 'p': 1}


[Parallel(n_jobs=8)]: Done 14250 out of 14250 | elapsed:  1.7min finished


In [29]:
'Gradient Boosting'
GB = GradientBoostingClassifier()
param_test2 = {
    'learning_rate': [0.5, 0.25, 0.1, 0.05, 0.01],
    'n_estimators':range(1,61,15),
    'max_depth':range(10,101,15),
    'min_samples_split':list(range(5, 31, 5)),
    'min_samples_leaf':range(20,80,20),
    'max_features':range(5,25,5)
}

GB_search = GridSearchCV(estimator = GB, param_grid = param_test2, n_jobs=8, refit = True, verbose = 3, cv=5)

# fitting the model for grid search 
GB_search.fit(X_train , y_train)
# GB_search.best_params_
# summarize
print('Mean Accuracy: %.3f' % GB_search.best_score_)
print('Config: %s' % GB_search.best_params_)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 10080 candidates, totalling 50400 fits


[Parallel(n_jobs=8)]: Done  18 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 144 tasks      | elapsed:   11.5s
[Parallel(n_jobs=8)]: Done 304 tasks      | elapsed:   21.6s
[Parallel(n_jobs=8)]: Done 528 tasks      | elapsed:   43.4s
[Parallel(n_jobs=8)]: Done 816 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done 1168 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done 1584 tasks      | elapsed:  2.9min
[Parallel(n_jobs=8)]: Done 2064 tasks      | elapsed:  3.6min
[Parallel(n_jobs=8)]: Done 2608 tasks      | elapsed:  4.9min
[Parallel(n_jobs=8)]: Done 3216 tasks      | elapsed:  6.0min
[Parallel(n_jobs=8)]: Done 3888 tasks      | elapsed:  7.4min
[Parallel(n_jobs=8)]: Done 4624 tasks      | elapsed:  8.9min
[Parallel(n_jobs=8)]: Done 5424 tasks      | elapsed: 10.5min
[Parallel(n_jobs=8)]: Done 6288 tasks      | elapsed: 12.3min
[Parallel(n_jobs=8)]: Done 7216 tasks      | elapsed: 14.4min
[Parallel(n_jobs=8)]: Done 8208 tasks      | elapsed: 16.2min
[Parallel(n_j

Mean Accuracy: 0.819
Config: {'learning_rate': 0.05, 'max_depth': 40, 'max_features': 15, 'min_samples_leaf': 40, 'min_samples_split': 10, 'n_estimators': 46}


# Oversampling

In [151]:
# X, y = data.values[:int(800),:-1], data.values[:int(800),-1]
# [print('Class {} has {} instances'.format(label, count))
#  for label, count in zip(*np.unique(y, return_counts=True))]

# kmeans_smote = KMeansSMOTE(
#     kmeans_args={
#         'n_clusters': 63
#     },
#     smote_args={
#         'k_neighbors': 10
#     }
# )
# X_resampled, y_resampled = kmeans_smote.fit_sample(X, y)

# [print('Class {} has {} instances after oversampling'.format(label, count))
#  for label, count in zip(*np.unique(y_resampled, return_counts=True))]
# print(Counter(y_resampled))

In [152]:
# define oversampling strategy
# oversample = RandomOverSampler(sampling_strategy='minority')

## Below code works for binary classification 
# oversample = RandomOverSampler(sampling_strategy=0.5)

In [153]:

# # define oversampling strategy
# oversample = RandomOverSampler(sampling_strategy='minority')

# # fit and apply the transform
# X_over, y_over = oversample.fit_resample(data.values[:,:-1], data.values[:,-1])

# Max = dict(Counter(y_over))[0.0]
# flag = True
# while flag:
#     X_over, y_over = oversample.fit_resample(X_over, y_over)
#     for x in list(dict(Counter(y_over)).values()):
#         if x == Max:
#             flag = False
#         else:
#             flag = True
#             break
        
# # summarize class distribution
# print(Counter(y_over))

In [154]:
# adict = dict()
# adict.update(zip(d.columns,col[:-1]))
# adict

In [155]:
# d = pd.DataFrame(X_over)
# adict = dict()
# adict.update(zip(d.columns,col[:-1]))
# d.rename(columns = adict, inplace =True)
# d['fclass'] = y_over
# d.head()

In [156]:
# count = Counter(data['fclass'])
# count

In [157]:
# data = d.copy()

In [42]:
cat0 = shuffle(data[data['fclass']==0])
cat1 = shuffle(data[data['fclass']==1])
cat2 = shuffle(data[data['fclass']==2])
cat5 = shuffle(data[data['fclass']==5])
cat6 = shuffle(data[data['fclass']==6])


In [43]:

lenCat0 = len(cat0.iloc[:,0])
lenCat1 = len(cat1.iloc[:,0])
lenCat2 = len(cat2.iloc[:,0])
lenCat5 = len(cat5.iloc[:,0])
lenCat6 = len(cat6.iloc[:,0])


In [44]:
methodList = []
accRes = []
accConf = []
nFold = 10
# sensitivityList = []
# specificityList = []

In [45]:
basicMethods = ['Gradient Boosting','Support Vector Machine', 'Logistic Regression', 'K Neighbors Classifier',\
                'Random Forest', 'Gaussian Naive Bayes', 'Linear Discriminant Analysis', 'Decision Tree']

methods = ['stacking classifir', 'soft votingClassifier','hard votingClassifier', 'Gradient Boosting',\
           'Support Vector Machine', 'Logistic Regression', 'K Neighbors Classifier', 'Random Forest',\
           'Gaussian Naive Bayes', 'Linear Discriminant Analysis', 'Decision Tree']
# methods = ['hard votingClassifier']

In [46]:
## 10 fold-CV

def classifiers(method):
    
    cl1 = LogisticRegression(C= 1000, max_iter= 700, penalty= 'l2', solver= 'sag')
    cl2 = KNeighborsClassifier(leaf_size=1, metric='manhattan', n_neighbors=18, p=1)
    cl3 = RandomForestClassifier(max_depth=40, max_features='sqrt', min_samples_leaf=4,\
                                 min_samples_split=4, n_estimators=31)
    cl4 = GaussianNB(var_smoothing= 1.0)
    cl5 = LinearDiscriminantAnalysis(shrinkage=0.03, solver = 'lsqr')
    cl6 = DecisionTreeClassifier(criterion='gini', max_depth=40, max_features=18,\
                                 min_samples_leaf=25, min_samples_split=4)
    cl7 = SVC(C=1000, gamma=0.01, kernel='rbf',probability=True)    
    cl8 = GradientBoostingClassifier(max_depth=55, max_features=5, min_samples_leaf=20,\
                                 min_samples_split=30, n_estimators=46)
    

    estimator = [(basicMethods[0],cl1), (basicMethods[1],cl2), (basicMethods[2],cl3),\
                 (basicMethods[3],cl4), (basicMethods[4],cl5), (basicMethods[5],cl6),\
                 (basicMethods[6],cl7), (basicMethods[7],cl8)]
    
    if method == 'Logistic Regression':
        CL = LogisticRegression(C= 1000, max_iter= 700, penalty= 'l2', solver= 'sag')
        
    if method == 'K Neighbors Classifier':
        CL = KNeighborsClassifier(leaf_size=1, metric='manhattan', n_neighbors=18, p=1)
        
    if method == 'Random Forest':
        CL = RandomForestClassifier(max_depth=40, max_features='sqrt', min_samples_leaf=4,\
                                 min_samples_split=4, n_estimators=31)
        
    if method == 'Gaussian Naive Bayes':
        CL = GaussianNB(var_smoothing= 1.0)
        
    if method == 'Linear Discriminant Analysis':
        CL = LinearDiscriminantAnalysis(shrinkage=0.03, solver = 'lsqr')
        
    if method == 'Decision Tree':
        CL = DecisionTreeClassifier(criterion='gini', max_depth=40, max_features=18,\
                                 min_samples_leaf=25, min_samples_split=4)
        
    if method == 'Support Vector Machine':
        CL = SVC(C=1000, gamma=0.01, kernel='rbf',probability=True)

    if method == 'Gradient Boosting':
        CL = GradientBoostingClassifier(max_depth=55, max_features=5, min_samples_leaf=20,\
                                 min_samples_split=30, n_estimators=46)
        
    if method == 'soft votingClassifier':
        CL = VotingClassifier(estimators=estimator, voting='soft',\
                              weights=[.096, 0.98, 0.7, 0.95, 0.96, 0.55, 0.7, 0.9])
        
    if method == 'hard votingClassifier':
        CL = VotingClassifier(estimators=estimator, voting='hard')
    
    if method == 'stacking classifir':
        CL = StackingClassifier(estimators=estimator, final_estimator=cl1, cv=5)
    print(type(CL))
    accList = []
    for j in range(nFold):

        i = j*.1
        k = (j+1)*.1
        #train
        X_train = cat0.iloc[int(k * lenCat0):,:-1].append(\
                  cat1.iloc[int(k * lenCat1):,:-1].append(\
                  cat2.iloc[int(k * lenCat2):,:-1].append(\
                  cat5.iloc[int(k * lenCat5):,:-1].append(\
                  cat6.iloc[int(k * lenCat6):,:-1].append(\
                  cat0.iloc[:int(i * lenCat0),:-1].append(\
                  cat1.iloc[:int(i * lenCat1),:-1].append(\
                  cat2.iloc[:int(i * lenCat2),:-1].append(\
                  cat5.iloc[:int(i * lenCat5),:-1].append(\
                  cat6.iloc[:int(i * lenCat6),:-1]                                       
                                                         )))))))))

        y_train = cat0.iloc[int(k * lenCat0):,-1].append(\
                  cat1.iloc[int(k * lenCat1):,-1].append(\
                  cat2.iloc[int(k * lenCat2):,-1].append(\
                  cat5.iloc[int(k * lenCat5):,-1].append(\
                  cat6.iloc[int(k * lenCat6):,-1].append(\
                  cat0.iloc[:int(i * lenCat0),-1].append(\
                  cat1.iloc[:int(i * lenCat1),-1].append(\
                  cat2.iloc[:int(i * lenCat2),-1].append(\
                  cat5.iloc[:int(i * lenCat5),-1].append(\
                  cat6.iloc[:int(i * lenCat6),-1]                                         
                                                        )))))))))
#         print(X_train.shape, y_train.shape)


        #preprocessing
        max_abs_scaler = preprocessing.MaxAbsScaler()
        X_train = max_abs_scaler.fit_transform(X_train)
        
        scaler = preprocessing.StandardScaler()
#         X_train = scaler.fit_transform(X_train)
 

        # fit and apply the Oversampling:
        #### Repeatitive oversampling
#         X_over, y_over = oversample.fit_resample(X_train, y_train)

#         Max = dict(Counter(y_over))[0.0]
#         flag = True
#         while flag:
#             X_over, y_over = oversample.fit_resample(X_over, y_over)
#             for x in list(dict(Counter(y_over)).values()):
#                 if x == Max:
#                     flag = False
#                 else:
#                     flag = True
#                     break
#         X_train, y_train = X_over.copy(), y_over.copy()
        # summarize class distribution
#         print(Counter(y_train))

#         print(X_train.shape, y_train.shape)
    

        # fit and apply the Oversampling:
        #### K-means oversampling
#         [print('Class {} has {} instances'.format(label, count))
#          for label, count in zip(*np.unique(y, return_counts=True))]

        kmeans_smote = KMeansSMOTE(
            kmeans_args={
                'n_clusters': 63
            },
            smote_args={
                'k_neighbors': 10
            }
        )
        X_train, y_train = kmeans_smote.fit_sample(X_train, y_train)

#         [print('Class {} has {} instances after oversampling'.format(label, count))
#          for label, count in zip(*np.unique(y_resampled, return_counts=True))]
        # summarize class distribution
#         print(Counter(y_train))

        #test
        X_test = cat0.iloc[int(i * lenCat0):int(k * lenCat0),:-1].append(\
                 cat1.iloc[int(i * lenCat1):int(k * lenCat1),:-1].append(\
                 cat2.iloc[int(i * lenCat2):int(k * lenCat2),:-1].append(\
                 cat5.iloc[int(i * lenCat5):int(k * lenCat5),:-1].append(\
                 cat6.iloc[int(i * lenCat6):int(k * lenCat6),:-1]))))

        y_test = cat0.iloc[int(i * lenCat0):int(k * lenCat0),-1].append(\
                 cat1.iloc[int(i * lenCat1):int(k * lenCat1),-1].append(\
                 cat2.iloc[int(i * lenCat2):int(k * lenCat2),-1].append(\
                 cat5.iloc[int(i * lenCat5):int(k * lenCat5),-1].append(\
                 cat6.iloc[int(i * lenCat6):int(k * lenCat6),-1]))))
#         print(X_test.shape, y_test.shape)
        X_test = max_abs_scaler.transform(X_test)
#         X_test = scaler.transform(X_test)   
        
        model = CL.fit(X_train, np.array(list(y_train)))
        
        predict = np.array(model.predict(X_test))
        
        cnf_matrix = metrics.confusion_matrix(y_test, predict)
        print(method)
        print(cnf_matrix)
#         #matplotlib inline
#         class_names=[0,1] # name  of classes
#         fig, ax = plt.subplots()
#         tick_marks = np.arange(len(class_names))
#         plt.xticks(tick_marks, class_names)
#         plt.yticks(tick_marks, class_names)

#         # create heatmap
#         sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
#         ax.xaxis.set_label_position("top")
#         plt.tight_layout()
#         plt.title(method+ ' fold of '+str(j), y=1.1)

#         plt.ylabel('Actual label')
#         plt.xlabel('Predicted label')

#         plt.show()


        acc = metrics.accuracy_score(y_test.values, predict)
        accList.append(acc)


    methodList.append(method)
    confidence = 0.95

    naccList = len(accList)
    maccList = np.mean(accList)
    std_erraccList = sem(accList)
    haccList = std_erraccList * t.ppf((1 + confidence) / 2, naccList - 1)




    print("Accuracy:         "+ str(round(maccList,2)) + '  -+' + str(round(haccList,4)))
    accRes.append(round(maccList,2))
    accConf.append(round(haccList,4))
    return (method, round(maccList,2), round(haccList,4))


In [ ]:
if __name__ == "__main__":
    p = Pool(processes=7)
    result1 = p.map(classifiers, methods)
    p.close()
    p.join()

<class 'sklearn.ensemble._voting.VotingClassifier'><class 'sklearn.ensemble._voting.VotingClassifier'><class 'sklearn.ensemble._gb.GradientBoostingClassifier'><class 'sklearn.svm._classes.SVC'><class 'sklearn.linear_model._logistic.LogisticRegression'><class 'sklearn.ensemble._stacking.StackingClassifier'>

<class 'sklearn.neighbors._classification.KNeighborsClassifier'>






/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.wa

/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.wa

/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 5. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 5. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserW

/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 5. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is 

...

/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.wa

<class 'sklearn.discriminant_analysis.LinearDiscriminantAnalysis'>


/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 2. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found

<class 'sklearn.tree._classes.DecisionTreeClassifier'>


/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 2. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found

In [ ]:
result = pd.DataFrame(result1)
result.columns = ['Method','Accuracy','ACC CI']

In [ ]:
result

In [ ]:
# # result = []
# result = pd.DataFrame(result1)
# result['Method'],result['Accuracy'],result['ACC CI'] = methodList,accRes,accConf
result.to_excel('kidney_Classification_Result_tune without oversampling.xlsx',index=False)
result